<a href="https://colab.research.google.com/github/dyou-wbg/meta_processing/blob/main/nostopwords_lemmnatization_LessOfDistance_Clean_50k_sample__char_based_distance_knn_deduping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
from google.colab import drive
import csv
import os
import re
import pandas as pd
import numpy as np

directory = '/content/drive/MyDrive/finalMetaFile'

Mounted at /content/drive


In [ ]:

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)


<ipython-input-2-1390d13c0cdd>:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [ ]:
# can skip if directly grab the 50k sample file
df = pd.read_csv('/content/drive/MyDrive/All_titles.csv')
df.rename(columns={'0': 'title'}, inplace=True)
df=df.drop(columns=['Unnamed: 0'])
df=df.sample(n=50000, random_state=321).dropna()

# deselect non-english titles

In [ ]:

import nltk
nltk.download('punkt')
from nltk import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# can skip if directly grab the 50k sample file
# select when title is at least >=2 character and have 70% of them in english
def remove_non_english(df):
  def is_english(text):
    words = word_tokenize(text)
    english_word_count = sum(len(word) >= 2 for word in words)
    return english_word_count / len(words) >= 0.7

  df_filtered = df[df['title'].apply(is_english)]
  return df_filtered

df_english_titles = remove_non_english(df.copy())

print("Original DataFrame:")
print(len(df))
print("\nDataFrame with English titles only:")
print(len(df_english_titles))

Original DataFrame:
49898

DataFrame with English titles only:
48173


In [ ]:
# can skip if directly grab the 50k sample file
df_english_titles = df_english_titles[~df_english_titles['title'].str.contains('و|д|я', regex=True)]

In [ ]:
len(df_english_titles)

47554

In [ ]:
df_english_titles = pd.read_csv('/content/drive/MyDrive/finalMetaFile/50k_sample.csv')

In [ ]:
df_english_titles=df_english_titles[~df_english_titles['title'].str.lower().isin(["conclusion", "poetry","introduction","front matter","editorial","back matter","correspondence","ist of Abbreviations","Book Reviews","Notifiable Diseases and Mortality Tables"])]
len(df_english_titles)

47526

In [ ]:
df_english_titles.to_csv(os.path.join(directory, '50k_sample.csv'), index=False)

In [ ]:
len(df_english_titles)/50000

0.95052

# from here

In [ ]:
df_english_titles = pd.read_csv('/content/drive/MyDrive/finalMetaFile/50k_sample.csv')

In [ ]:
df_english_titles['title'] = df_english_titles['title'].str.lower()

In [ ]:
len(df_english_titles)

47526

In [ ]:
df_english_titles.drop_duplicates(subset='title', keep='first')

,title
0,hazel raises funds to take produce protection ...
1,determinants of risk sharing through remittances
2,huckettomyia
3,the mainren model for sustainable built enviro...
4,poverty and disability: the endless loop
...,...
47521,diversity in the labour market: the legal fram...
47522,financial flows and exchange rates: challenges...
47523,should crisis cartels exist amid crises?
47524,transnational corporations and local firms in ...


In [ ]:
df_english_titles.drop_duplicates()

,title
0,hazel raises funds to take produce protection ...
1,determinants of risk sharing through remittances
2,huckettomyia
3,the mainren model for sustainable built enviro...
4,poverty and disability: the endless loop
...,...
47521,diversity in the labour market: the legal fram...
47522,financial flows and exchange rates: challenges...
47523,should crisis cartels exist amid crises?
47524,transnational corporations and local firms in ...


In [ ]:
print("===========================")
duplicate_rows = df_english_titles[df_english_titles.duplicated(subset='title')]
print(duplicate_rows)

print("===========================")
len(duplicate_rows)/len(df_english_titles)

                                                   title
3176                                        book reviews
3631                                        book reviews
3991                                      calcutta diary
5076                                 work and employment
6105                                   england and wales
...                                                  ...
45362  on the stability of preferences: repercussions...
46094                                chronique politique
46576                      chapter ii. the world economy
46617                mixture models of choice under risk
46775                           current labor statistics

[149 rows x 1 columns]


0.0031351260362748813

In [ ]:
df_english_titles_temp = df_english_titles.drop(duplicate_rows.index)

In [ ]:
len(df_english_titles_temp)/len(df_english_titles)

0.9973277784791482

In [ ]:
df_english_titles=df_english_titles_temp

In [ ]:
!pip install nltk

In [ ]:
import nltk
nltk.download('stopwords')
stop_words = set(nltk.corpus.stopwords.words('english'))
df_english_titles['title'] = df_english_titles['title'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:

nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
df_english_titles['title'] = df_english_titles['title'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))


[nltk_data] Downloading package wordnet to /root/nltk_data...


# fuzzy matching

In [ ]:
from string import printable
from collections import Counter

printable_dict = {c:i for i,c in enumerate(printable)}

def vectorize(title):
    cnt = Counter(title)
    array = [0 for _ in printable]+[0] # Last position is for nonprintable characters
    for c,n in cnt.items():
        if c in printable_dict:
            array[printable_dict[c]]=n
        else:
            array[-1]+=n # Update last position if it's not printable
    return np.array(array)

vects = []
for title in df_english_titles.title:
    vects.append(vectorize(title))

In [ ]:
%%time
threshold = 5
min_length = 20
# Iterate over title pairs
for i, v1 in enumerate(vects):
    if sum(v1[:-1]) > min_length: # Only above a minimum number of printable characters
        for j, v2 in enumerate(vects):
            if sum(v2[:-1]) > min_length:
                if i!=j:
                    l1 = np.linalg.norm((v1[:-1] - v2[:-1]), ord=1)
                    if l1 < threshold:
                        t1 = df_english_titles.iloc[i].title
                        t2 = df_english_titles.iloc[j].title
                        print(t1)
                        print(t2)
                        print('_'*100)


CENTRAL EUROPEAN REVIEW OF ECONOMICS AND MANAGEMENT
CENTRAL EUROPEAN REVIEW OF ECONOMICS AND MANAGEMENT
____________________________________________________________________________________________________
Relative Wage Variability: Monetary Policy and the Labor Market
Relative Wage Variability: Monetary Policy and the Labor Market
____________________________________________________________________________________________________
Editor's introduction
Editor's Introduction
____________________________________________________________________________________________________
Editor's introduction
Editor's Introduction
____________________________________________________________________________________________________
Current Labor Statistics
Current Labor Statistics
____________________________________________________________________________________________________
Current Labor Statistics
Current Labor Statistics
___________________________________________________________________________

# use this to drop all exact match cases

In [ ]:
threshold = 5
min_length = 20

seen_pairs = set()  # Store seen title pairs to avoid duplicates

# Iterate over title pairs
for i, v1 in enumerate(vects):
    if sum(v1[:-1]) > min_length:
        for j, v2 in enumerate(vects):
            if sum(v2[:-1]) > min_length and i != j:
                l1 = np.linalg.norm((v1[:-1] - v2[:-1]), ord=1)

                if l1 < threshold:
                    title_pair = tuple(sorted([df_english_titles.iloc[i].title, df_english_titles.iloc[j].title]))  # Sort alphabetically
                    if title_pair not in seen_pairs:  # Check for duplicates
                        seen_pairs.add(title_pair)  # Add to seen pairs
                        t1, t2 = title_pair  # Unpack titles
                        print(t1)
                        print(t2)
                        print('_' * 100)

                        #print(f"{t1} & {t2} (score: {l1:.4f})")


could corporation become engine international cooperation
could corporation become engine international cooperation?
____________________________________________________________________________________________________
economic aspect autism
economics autism egypt
____________________________________________________________________________________________________
introduction constitution india
introduction constitution india
____________________________________________________________________________________________________
risk management financial derivatives: overview
risk management financial derivatives:an overview
____________________________________________________________________________________________________


In [ ]:
threshold = 5
min_length = 20

seen_pairs = set()  # Store seen title pairs to avoid duplicates

# Iterate over title pairs
for i, v1 in enumerate(vects):
    if sum(v1[:-1]) > min_length:
        title1 = df_english_titles.iloc[i].title
        # Check for minimum word and character count
        if len(title1.split()) >= 2 and len(title1) >= 10:
            for j, v2 in enumerate(vects):
                if sum(v2[:-1]) > min_length and i != j:
                    l1 = np.linalg.norm((v1[:-1] - v2[:-1]), ord=1)

                    if l1 < threshold:
                        title2 = df_english_titles.iloc[j].title
                        # Check for minimum word and character count
                        if len(title2.split()) >= 2 and len(title2) >= 10:
                            title_pair = tuple(sorted([title1, title2]))  # Sort alphabetically
                            if title_pair not in seen_pairs:  # Check for duplicates
                                seen_pairs.add(title_pair)  # Add to seen pairs
                                t1, t2 = title_pair
                                print(f"{t1} & {t2} (score: {l1:.4f})")
                                print('_' * 100)

could corporation become engine international cooperation & could corporation become engine international cooperation? (score: 1.0000)
____________________________________________________________________________________________________
economic aspect autism & economics autism egypt (score: 4.0000)
____________________________________________________________________________________________________
introduction constitution india & introduction constitution india (score: 0.0000)
____________________________________________________________________________________________________
risk management financial derivatives: overview & risk management financial derivatives:an overview (score: 2.0000)
____________________________________________________________________________________________________
arrival fast internet employment africa & arrival fast internet employment africa â (score: 1.0000)
__________________________________________________________________________________________________